<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/adapters_classification_no_pretraining__continue_training_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
This tests using an adapter for pretraining

# Things to change in real run
* Use the proper datasets (not dev for validation?)




# Setup

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
%cd '/content/drive/MyDrive/AdaptOrDie'

In [ ]:
!pip install datasets
!pip install transformers[torch]
!pip install adapters
!pip install scikit-learn

In [ ]:
!pip install pynvml

In [ ]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    gpu_used = info.used//1024**2
    print(f"GPU {gpu_used} MB")
    return gpu_used

print_gpu_utilization()

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'f1_macro': f1
    }

In [ ]:
from datasets import load_dataset

dataset_name = "BigTMiami/amazon_helpfulness"
training_name = "0_to_5_percent"
train_dataset = load_dataset(dataset_name, split="train[:5%]")
# train_2 = load_dataset(dataset_name, split="train[5%:10%]")
# train_3 = load_dataset(dataset_name, split="train[10%:20%]")
# train_4 = load_dataset(dataset_name, split="train[20%:40%]")
# train_5 = load_dataset(dataset_name, split="train[40%:]")
validation_dataset = load_dataset(dataset_name, split="dev")

In [ ]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)
print_gpu_utilization()

In [ ]:
# adapter_hub_name = "adapter_HUB_classification_no_pretraining_test"
# adapter_name = "adapter_classification_seq_bn_no_pretraining"
# adapter_type = "seq_bn" # could be "lora", etc.

# # Add a new adapter
# model.add_adapter(adapter_name, config=adapter_type)

# # Add head for masked language modeling
# model.add_classification_head(
#     adapter_name,
#     num_labels=2,
#     id2label={ 0: "unhelpful", 1: "helpful"}
#   )

# # Set the adapter to be used for training
# model.train_adapter(adapter_name)


In [ ]:
adapter_hub_name = f"adapter_classification_no_pretraining_continue_{training_name}_test"

adapter_name = model.load_adapter("BigTMiami/adapter_HUB_classification_no_pretraining_test")

# Set the adapter to be used for training
model.train_adapter(adapter_name)

print(adapter_name)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
print_gpu_utilization()

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Train

In [ ]:
from transformers import TrainingArguments
from adapters import AdapterTrainer

training_args = TrainingArguments(
    output_dir=f"adapter_classifier_training_{training_name}",
    learning_rate=1e-4,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=100,
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
training_results = trainer.train()
print(training_results)

In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

In [ ]:
print_gpu_utilization()

In [ ]:
all_results[0]["train"]
all_results[0]["train"][1]
all_results[0]["train"][2]["train_runtime"]
all_results[0]["eval"]["eval_loss"]

In [ ]:
training_loss = training_results[1]
training_time = training_results["train_runtime"]
eval_loss = eval_results["eval_loss"]
eval_f1_macro = 100.0 * eval_results["eval_f1_macro"]
eval_accuracy = 100.0 * eval_results["eval_accuracy"]
print(f"{i}: Eval F1: {eval_f1_macro:.3f} Acc: {eval_accuracy:.3f} Loss: {eval_loss:.5f} || Training Loss: {training_loss:.4f} Time: {training_time:.0f}")

In [ ]:
trainer.state.log_history

In [ ]:
import pandas as pd
pd.DataFrame(trainer.state.log_history)

In [ ]:
model.push_adapter_to_hub(
    adapter_hub_name,
    adapter_name,
    datasets_tag=dataset_name
)

In [ ]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()